<a href="https://colab.research.google.com/github/ProfDoof/advent_of_code/blob/2022/day8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 

## Load Data

In [1]:
from pathlib import Path

DAY = 8
DATA_FILE = Path.cwd() / 'drive' / 'MyDrive' / 'AdventOfCode' / 'aoc_data' / f'day{DAY}.txt'

data = DATA_FILE.read_text()

test = '''
30373
25512
65332
33549
35390
'''

## Solution

In [26]:
from typing import List, Optional
from enum import Enum, auto
import pprint
import itertools

tree_heights: List[List[int]] = []

for line in data.strip().split('\n'):
    tree_heights.append([])
    for tree in line:
        tree_height = int(tree)
        tree_heights[-1].append(tree_height)

n_max_trees: List[List[Optional[int]]] = []
e_max_trees: List[List[Optional[int]]] = []
s_max_trees: List[List[Optional[int]]] = []
w_max_trees: List[List[Optional[int]]] = []

for trees in tree_heights:
    n_max_trees.append([])
    e_max_trees.append([])
    s_max_trees.append([])
    w_max_trees.append([])
    for tree in trees:
        n_max_trees[-1].append(None)
        e_max_trees[-1].append(None)
        s_max_trees[-1].append(None)
        w_max_trees[-1].append(None)


class Direction(Enum):
    North = auto()
    East = auto()
    South = auto()
    West = auto()


def __get_max_height(kb: List[List[Optional[int]]], row: int, col: int, dir: Direction):
    if row < 0 or row >= len(tree_heights) or col < 0 or col >= len(tree_heights[row]):
        return -1

    if kb[row][col] is None:
        max_height = get_max_height_in_dir(row, col, dir)
        kb[row][col] = max(tree_heights[row][col], max_height)
    
    return kb[row][col]

def get_max_height_in_dir(row: int, col: int, dir: Direction):
    if dir == Direction.North:
        return __get_max_height(n_max_trees, row - 1, col, dir)
    elif dir == Direction.South:
        return __get_max_height(s_max_trees, row + 1, col, dir)
    elif dir == Direction.East:
        return __get_max_height(e_max_trees, row, col + 1, dir)
    elif dir == Direction.West:
        return __get_max_height(w_max_trees, row, col - 1, dir)


visible_trees: List[List[str]] = []

for row_idx, trees in enumerate(tree_heights):
    visible_trees.append([])
    for col_idx, tree in enumerate(trees):
        n_max_height = get_max_height_in_dir(row_idx, col_idx, Direction.North)
        e_max_height = get_max_height_in_dir(row_idx, col_idx, Direction.East)
        s_max_height = get_max_height_in_dir(row_idx, col_idx, Direction.South)
        w_max_height = get_max_height_in_dir(row_idx, col_idx, Direction.West)

        if any([tree > h for h in [n_max_height, e_max_height, s_max_height, w_max_height]]):
            visible_trees[-1].append('V')
        else:
            visible_trees[-1].append('.')

num_vis_trees = len(list(filter(lambda x: x == 'V', itertools.chain.from_iterable(visible_trees))))
print(f'There are {num_vis_trees} visible trees')

def get_tree_in_dir(row: int, col: int, dir: Direction):
    if dir == Direction.North:
        row = row - 1
    elif dir == Direction.South:
        row = row + 1
    elif dir == Direction.East:
        col = col + 1
    elif dir == Direction.West:
        col = col - 1
    if row < 0 or col < 0:
        raise IndexError()

    return row, col, tree_heights[row][col]


def get_view_score_in_dir(row: int, col: int, dir: Direction):
    height = tree_heights[row][col]
    view = 0
    while True:
        try:
            row, col, dir_tree = get_tree_in_dir(row, col, dir)
            if dir_tree >= height:
                view += 1
                break
    
            view += 1
        except IndexError:
            break
    return view

highest_scenic_score = 0
scenic_scores = []

for row_idx, trees in enumerate(tree_heights):
    scenic_scores.append([])
    for col_idx, tree in enumerate(trees):
        n_view = get_view_score_in_dir(row_idx, col_idx, Direction.North)
        e_view = get_view_score_in_dir(row_idx, col_idx, Direction.East)
        s_view = get_view_score_in_dir(row_idx, col_idx, Direction.South)
        w_view = get_view_score_in_dir(row_idx, col_idx, Direction.West)

        highest_scenic_score = max(highest_scenic_score, n_view * e_view * s_view * w_view)
        scenic_scores[-1].append(n_view * e_view * s_view * w_view)

# pprint.pp(scenic_scores)
print(f'The highest possible scenic score is {highest_scenic_score}')

There are 1543 visible trees
The highest possible scenic score is 595080
